# Определение количества ботов на сайте

##### Подготовка. 
Получаем данные с помощью get-запроса и формирования параметров. Найдём процент ботов от общего числа визитов за сентябрь 2020 года, с группировками по источнику и времени.

In [1]:
import json
import requests
from pprint import pprint
import pandas as pd

token = 'AQAAAAA45CCcAAedM6tbmmuHP0IChtThvrF9Ql4'
headers = {'Authorization':'OAuth' + token} 
params = {'metrics': 'ym:s:visits,ym:s:robotPercentage',
          'dimensions': 'ym:s:lastsignTrafficSource,ym:s:hour',
          'date1': '2020-09-01',
          'date2': '2020-09-30',
          'limit':100000,
          'ids': 30177909,
          'accuracy':'full'}
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
print (response.status_code)
metrica_data = response.json()

200


Убедились, что данные по запросу нам доступны.
Теперь напишем функцию, которая поможет перевести наши данные в ДатаФрейм.
Функция создаст список словарей.

In [2]:
def getMetrikaDataInListOfDict(metrica_data):
    list_of_dict = []
    dimensions_list = metrica_data['query']['dimensions'] # Список группировок которые мы запрашивали
    metrics_list = metrica_data['query']['metrics'] # Список метрик которые мы запрашивали
    for data_item in metrica_data['data']: # Обращаемся уже к данным, список данных отправляем в переменную
        d = {} #  Пустой словарь
        for i, dimensions in enumerate(data_item['dimensions']):
            d[dimensions_list[i]] = dimensions['name'] # Отправляем данные в словарь, ключ - номер, значение -
            # список значений наших группировок
        for i, metric in enumerate(data_item['metrics']):
            d[metrics_list[i]] = metric
        list_of_dict.append(d)
    return list_of_dict

##### Работа с Датафреймом
Обработаем данные с помощью функции, переведём в ДатаФрейм.
Добавим столбец, который считает количество роботов.

In [3]:
metrika_list_if_dict = getMetrikaDataInListOfDict(metrica_data) # Получился список словарей
metrica_df = pd.DataFrame(metrika_list_if_dict)
metrica_df.columns = ['Source', 'Hour', 'Visits', 'RobotPercent']
metrica_df['RobotCount'] = round(metrica_df['Visits'] / 100 * metrica_df['RobotPercent'])
print(metrica_df.head(5))

                  Source   Hour  Visits  RobotPercent  RobotCount
0  Search engine traffic  16:00   251.0     24.701195        62.0
1  Search engine traffic  15:00   240.0     26.250000        63.0
2  Search engine traffic  11:00   228.0     25.877193        59.0
3  Search engine traffic  13:00   227.0     28.193833        64.0
4  Search engine traffic  12:00   225.0     24.888889        56.0


#### Задача 1. Оценить количество ботов, которые заходят на наш сайт, и их долю от всего трафика.

In [4]:
robot_total = metrica_df['RobotCount'].sum()
print('Общее количество роботов, посетивших сайт в сентябре 2020 года: {}'.format(robot_total))
robot_total_percent = round((robot_total / metrica_df['Visits'].sum()), 4)
print('Процент роботов от общего числа визитов на сайт в сентябре 2020 года: {}%'.format(robot_total_percent*100))

Общее количество роботов, посетивших сайт в сентябре 2020 года: 962.0
Процент роботов от общего числа визитов на сайт в сентябре 2020 года: 22.29%


#### Задача 2. Определить, с каких источников и в какое время суток чаще всего заходят роботы.

##### 2.1. Определяем источник с самым большим количеством роботов.

In [6]:
metrica_df_source = metrica_df.groupby(['Source'])['RobotCount'].sum().sort_values(ascending=False)
max_robot_source = metrica_df_source[metrica_df_source == metrica_df_source.max()].index[0]
print('Источник, из которого чаще всего роботы заходят на сайт: {}'.format(max_robot_source))

Источник, из которого чаще всего роботы заходят на сайт: Search engine traffic


##### 2.2. Определяем час, в который заходит наибольшее количество роботов.

In [7]:
metrica_df_time = metrica_df.groupby(['Hour'])['RobotCount'].sum().sort_values(ascending=False)
max_robot_time = metrica_df_time[metrica_df_time == metrica_df_time.max()].index[0]
print('Время, в которое чаще всего роботы заходят на сайт: {}'.format(max_robot_time))

Время, в которое чаще всего роботы заходят на сайт: 13:00


##### 2.3. Определяем время суток, в которое заходит наибольшее количество роботов.

Для этого переведём часы в формат времени, извлечём час. И создатим доплнительный столбец с временем суток с помощью функции.

In [8]:
metrica_df['Hour'] = pd.to_datetime(metrica_df['Hour']).dt.hour
def get_daytime(hour):
    if hour >= 0 and hour <= 6:
        return 'night'
    if hour > 6 and hour <= 12:
        return 'morning'
    if hour > 12 and hour <= 18:
        return 'day'
    if hour > 18 and hour < 24:
        return 'evening'
    
metrica_df['Daytime'] = metrica_df['Hour'].apply(get_daytime)
metrica_df_daytime = metrica_df.groupby(['Daytime'])['RobotCount'].sum().sort_values(ascending=False)
max_robot_daytime = metrica_df_daytime[metrica_df_daytime == metrica_df_daytime.max()].index[0]
print('Время суток, в которое чаще всего роботы заходят на сайт: {}'.format(max_robot_daytime))

Время суток, в которое чаще всего роботы заходят на сайт: day


### Выводы.
#### 1. Из всех посещений в сентябре 2020 года более 22% пришлось на роботов
#### 2. Роботы чаще всего заходят из источника "Search engine traffic"
#### 3. Роботы чаще всего заходят в дневное время суток, наиболее часто - около 13:00